### Sep

In [1]:
import pandas as pd
import json
from collections import defaultdict as ddict
from datetime import date
import lexical_features as lf
import brandseg
from tqdm.notebook import tqdm

bs = brandseg.BrandSeg()

In [2]:
domains_vt1 = '/bad_ip_data/ucsc/Vaccine_Disinfo/data/covid_resources/dataset_sep_2020/oxford-vaccine-article_urls-shared-on-twitter.csv.1.scan.report'
domains_vt2 = '/bad_ip_data/ucsc/Vaccine_Disinfo/data/covid_resources/dataset_sep_2020/oxford-vaccine-article_urls-shared-on-twitter.csv.2.scan.report'

In [3]:
domains_vt = [i.split('\n')[0] for i in open(domains_vt1,'r').readlines()]+[i.split('\n')[0] for i in open(domains_vt2,'r').readlines()]

In [4]:
def timeduration(d0,d1):
    d0y,d0m,d0d =  list(map(int,d0.split(' ')[0].split('-')))
    d1y,d1m,d1d =  list(map(int,d1.split(' ')[0].split('-')))
    return (date(d1y,d1m,d1d)-date(d0y,d0m,d0d)).days

In [5]:
import tldextract
VT = ddict(list)
for i in tqdm(domains_vt):
    try:
        tmp = json.loads(i)
        url = tldextract.extract(tmp.get('url'))
        VT['domain'].append('{}.{}'.format(url.domain,url.suffix))
        VT['scanned url'].append(tmp.get('url'))
        VT['total'].append(tmp.get('total'))
        VT['positives'].append(tmp.get('positives'))
        VT['duration[VT]'].append(timeduration(tmp.get('first_seen'),tmp.get('last_seen')))
        
        URL_FINAL = tmp.get('url')
        
        try:
            rlength = tmp.get('additional_info').get('rlength')
            if rlength != None : VT['rlength'].append(rlength)
            if rlength == None : VT['rlength'].append(None)
        except AttributeError:
            VT['rlength'].append(None)
        
        try:
            resolution = tmp.get('additional_info').get('resolution')
            if resolution != None : VT['resolution'].append(resolution)
            if resolution == None : VT['resolution'].append(None)
        except AttributeError:
            VT['resolution'].append(None)
            
        try:    
            server = tmp.get('additional_info').get('Response headers').get('server')
            if server != None: VT['server'].append(server)
            if server == None: VT['server'].append(None)
        except AttributeError:
            VT['server'].append(None)
        
        try:
            URLAfterRredirects = tmp.get('additional_info').get('URL after redirects')
            if URLAfterRredirects != None: 
                VT['URL after redirects'].append(URLAfterRredirects)
                VT['Final URL'].append(URLAfterRredirects)
                URL_FINAL = URLAfterRredirects
            if URLAfterRredirects == None:
                VT['URL after redirects'].append('None')
                VT['Final URL'].append(tmp.get('url'))
        except AttributeError:
            VT['URL after redirects'].append(None)
            VT['Final URL'].append(tmp.get('url'))
            
        lex = lf.get_features(URL_FINAL, bs)
            
        VT['suspicious_tld'].append(lex.get('suspicious_tld'))
        VT['length'].append(lex.get('length'))
        VT['fake_tld'].append(lex.get('fake_tld'))
        VT['brand'].append(lex.get('brand'))
        VT['pop_keywords'].append(lex.get('pop_keywords'))
        VT['similar'].append(lex.get('similar'))
        VT['is_idn'].append(lex.get('is_idn'))
        VT['minus'].append(lex.get('minus'))
        VT['num_subdomains'].append(lex.get('num_subdomains'))
            
    except:
        pass
VT = pd.DataFrame(VT)

In [6]:
VT

,domain,scanned url,total,positives,duration[VT],rlength,resolution,server,URL after redirects,Final URL,suspicious_tld,length,fake_tld,brand,pop_keywords,similar,is_idn,minus,num_subdomains
0,forbes.com.br,https://forbes.com.br/negocios/2020/06/astraze...,83,0,0,143266.0,104.26.8.58,cloudflare,None,https://forbes.com.br/negocios/2020/06/astraze...,N,100,0,0,0,0,N,8,1
1,google.com,https://www.google.com/amp/s/www.elespanol.com...,83,0,0,77255.0,173.194.193.106,nginx,https://www.elespanol.com/invertia/observatori...,https://www.elespanol.com/invertia/observatori...,N,142,0,0,0,0,N,6,2
2,xhne.ws,http://xhne.ws/yXuub,83,0,0,23767.0,203.192.12.6,Tengine,http://www.xinhuanet.com/english/2020-09/02/c_...,http://www.xinhuanet.com/english/2020-09/02/c_...,N,59,0,0,0,0,N,1,2
3,nbcnews.com,https://www.nbcnews.com/health/health-news/lar...,83,0,0,276310.0,23.59.20.89,nginx,None,https://www.nbcnews.com/health/health-news/lar...,N,98,0,0,0,1,N,10,1
4,mol.im,https://mol.im/a/8684869%20http:/dailym.ai/and...,83,0,98,174.0,23.48.106.183,None,None,https://mol.im/a/8684869%20http:/dailym.ai/and...,N,50,0,0,0,0,N,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14831,indianexpress.com,https://indianexpress.com/article/india/oxford...,83,0,0,44593.0,23.60.136.94,nginx,None,https://indianexpress.com/article/india/oxford...,N,184,0,0,0,0,N,10,2
14832,launica945.com,https://launica945.com/category/meche-castro-e...,83,0,0,109286.0,66.96.147.113,Apache/2,https://www.launica945.com/category/meche-cast...,https://www.launica945.com/category/meche-cast...,N,61,0,0,0,0,N,4,1
14833,twitter.com,https://twitter.com/SMadlokovu/status/13021590...,83,0,0,2251.0,104.244.42.129,tsa_devel,https://mobile.twitter.com/SMadlokovu/status/1...,https://mobile.twitter.com/SMadlokovu/status/1...,N,64,0,1,0,1,N,0,1
14834,cnb.cx,https://cnb.cx/3hjLheQ,83,0,0,1557001.0,67.199.248.13,None,https://www.cnbc.com/2020/09/10/who-calls-dela...,https://www.cnbc.com/2020/09/10/who-calls-dela...,N,304,0,0,0,1,N,12,2
